In [1]:
from datascience import *
from datascience.predicates import are
path_data = '../../../../data/'
import numpy as np
import matplotlib
matplotlib.use('Agg')
%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)

from urllib.request import urlopen 
import re
from bancor3_simulator.protocol import BancorV3
from bancor3_simulator.core.settings import GlobalSettings
from decimal import Decimal
import pandas as pd

def read_url(url): 
    return re.sub('\\s+', ' ', urlopen(url).read().decode())

# Bootstrapping Behavior

To illustrate how the system works, we will constrain the system to an oversimplified case with only three pools: ETH, wBTC and LINK, in addition to the BNT omnipool. Assume that these pools have just recently earned whitelist status, they each have a BNT funding limit of 40,000 BNT, and all are completely empty.

In [2]:
import pandas as pd
from decimal import Decimal
from bancor3_simulator.protocol import BancorV3

In [3]:
whitelisted_tokens: list = ['bnt', 'eth', 'wbtc', 'link']

In [4]:
bnt_funding_limit: Decimal = Decimal('40000')

The protocol is in the “bootstrapping phase” for all three assets. Users can still interact with the system and deposit liquidity - which is essential for the process. The system needs to attract sufficient quantities of each token to overcome the minimum liquidity threshold prior to the activation of each liquidity pool, respectively. The default minimum liquidity threshold is 10,000 BNT (previously 1,000 BNT in an earlier draft of this proposal), meaning that the pool must mint 10,000 BNT during its activation, and therefore there must be at least a commensurate value of TKN available in the vault.

In [5]:
bnt_min_liquidity: Decimal = Decimal('10000')

Assume that the prices of each asset are as follows: BNT = 2.50, LINK = 15.00, ETH = 2,500.00, wBTC = 40,000.00. Therefore, to bootstrap each pool, at least $25,000 of each asset must be available in the vault.

In [6]:
# Recall from an earlier chapter that we already defined the cooldown period and the exit fees as follows.
cooldown_time: int = 7
withdrawal_fee: Decimal = Decimal('0.0025')
price_feeds = pd.DataFrame({'INDX':[0],
                            'bnt':[2.5],
                            'link':[15.00],
                            'eth':[2500.00],
                            'wbtc':[40000.00]})

network_fee: Decimal = Decimal('0.2')
trading_fee: Decimal = Decimal('0.01')

# There are other possible configuration settings available, however for the present purpose we will use the defaults.
v3 = BancorV3(whitelisted_tokens=whitelisted_tokens,
              network_fee=network_fee,
              trading_fee=trading_fee,
              cooldown_time=cooldown_time,
              withdrawal_fee=withdrawal_fee,
              bnt_min_liquidity=bnt_min_liquidity,
              bnt_funding_limit=bnt_funding_limit,
              price_feeds=price_feeds)

v3.global_state

State(latest_tkn=None, latest_amt=Decimal('0'), latest_user_name=None, latest_action=None, withdrawal_ids=[], created_at=datetime.datetime(2022, 5, 16, 9, 21, 39, 781311), timestep=0, network_fee=Decimal('0.2'), trading_fee=Decimal('0.01'), bnt_min_liquidity=Decimal('10000'), cooldown_time=7, bnt_funding_limit=Decimal('40000'), alpha=Decimal('0.200000000000000000'), whitelisted_tokens=['bnt', 'eth', 'wbtc', 'link'], staked_bnt=Decimal('0'), vault_bnt=Decimal('0'), erc20contracts_bnbnt=Decimal('0'), protocol_wallet_bnbnt=Decimal('0'), vortex_bnt=Decimal('0'), vortex_vbnt_burned=Decimal('0'), tokens=defaultdict(<class 'bancor3_simulator.core.dataclasses.Transaction'>, {'bnt': Transaction(timestep=0, vault_tkn=Decimal('0'), erc20contracts_bntkn=Decimal('0'), staked_tkn=Decimal('0'), trading_enabled=False, bnt_trading_liquidity=Decimal('0'), tkn_trading_liquidity=Decimal('0'), trading_fee=Decimal('0.01'), bnt_funding_limit=Decimal('40000'), bnt_remaining_funding=Decimal('0'), bnt_funding_a

During the bootstrap phase, Alice deposits 100 ETH, Bob deposits 100 bnwBTC, and Charlie deposits 10,000 LINK. By default, the initial pool token value is 1:1 with the underlying asset, and only changes after revenue begins accumulating. Therefore, each of these characters receive 100 bnETH, 100 bnwBTC and 1,000 bnLINK pool tokens for their contribution, respectively,

In [7]:
v3.create_user('Alice')
v3.create_user('Bob')
v3.create_user('Charlie')

However, note that the trading liquidity available on the pools remains unchanged until the BancorDAO msig signers initialize each pool.

The condition to initialize a pool is that there must be at least 10,000 BNT ($25,000 at the quoted price) worth of each token - in this case all three satisfy the criteria. The BancorDAO msig signers then sign a transaction with the rate of BNT/TKN for each of the bootstrapping assets. In this case, BNT/ETH = 1,000, BNT/wBTC = 16,000, BNT/LINK =  6. The trading liquidity on each pool begins with exactly 10,000 BNT and the commensurate amount of TKN as determined by the rates supplied by the msig signers.

It is important to note here that each of these pools have been stated to have a 100,000 BNT funding limit, 10× higher than the bootstrapping depth. This is because the pools should equilibrate with the rest of the market and grow in a controlled manner. Therefore, only a fraction of the available ETH, wBTC and LINK has been committed to trading at this stage. The other important change is that the BNT that was created by the protocol causes its own pool token to be issued, bnBNT,

Fortunately, the rate at which the pool can grow is independent of the new liquidity additions, and is determined entirely by the token balances available in the vault. To demonstrate, suppose that each user deposits just 1 more TKN each to their respective pools; Alice deposits 1 ETH, Bob deposits 1 wBTC, and Charlie deposits 1 LINK. Each of these tokens is accepted directly to the vault. Assume that no trading has yet occurred on the pool, therefore the valuation of each pool token remains at 1:1 for the time being. Therefore, each user is issued one additional bnTKN.

In [8]:
# The actual deposits.

v3.stake(tkn_amt=100, tkn_name='eth', user_name='Alice')

v3.stake(tkn_amt=10000, tkn_name='link', user_name='Charlie')

v3.stake(tkn_amt=100, tkn_name='wbtc', user_name='Bob')

v3.describe()

,Trading Liquidity,Vault,Staking,ERC20 Contracts,Vortex,External Protection,Protocol Wallet
0,,bnt=0.000000,bnt=0.000000,bnbnt=0.000000,bnt=0.000000,bnt=0.000000,bnbnt=0.000000
1,bnt=0.000000 eth=0.000000,eth=100.000000,eth=100.000000,bneth=100.000000,,eth=0.000000,
2,bnt=0.000000 wbtc=0.000000,wbtc=100.000000,wbtc=100.000000,bnwbtc=100.000000,,wbtc=0.000000,
3,bnt=0.000000 link=0.000000,link=10000.000000,link=10000.000000,bnlink=10000.000000,,link=0.000000,
4,,,,,,,


In [9]:
v3.stake(tkn_amt=1, tkn_name='eth', user_name='Alice')

v3.stake(tkn_amt=1, tkn_name='link', user_name='Charlie')

v3.stake(tkn_amt=1, tkn_name='wbtc', user_name='Bob')

v3.dao_msig_init_pools(whitelisted_tokens)

v3.describe()

,Trading Liquidity,Vault,Staking,ERC20 Contracts,Vortex,External Protection,Protocol Wallet
0,,bnt=60000.000000,bnt=60000.000000,bnbnt=60000.000000,bnt=0.000000,bnt=0.000000,bnbnt=60000.000000
1,bnt=20000.000000 eth=20.000000,eth=101.000000,eth=101.000000,bneth=101.000000,,eth=0.000000,
2,bnt=20000.000000 wbtc=1.250000,wbtc=101.000000,wbtc=101.000000,bnwbtc=101.000000,,wbtc=0.000000,
3,bnt=20000.000000 link=3333.333333,link=10001.000000,link=10001.000000,bnlink=10001.000000,,link=0.000000,
4,,,,,,,


and the balances of the staking ledger are updated accordingly.